<a href="https://colab.research.google.com/github/patriciamedyna/Classifier_page/blob/main/Fastai_with_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *

In [4]:
path = '/content/gdrive/MyDrive/Colab Notebooks/Tobacco_LSTM/'
files = glob.glob(path + 'txt/*.txt')
len(files)

1290

In [5]:
# obtem a classe (numero de paginas) do arquivo
def get_target(filepath):
    fname = filepath.split('/')[-1]
    if '_' in fname:
        return 1
        
    return 0

In [6]:
def extract_content(filepath):
    data = []
    with open(filepath, 'r') as fp:
        data = fp.readlines()
            
    return data

In [7]:
texts = []
target = []
for i, fp in enumerate(files):
  target.append(get_target(fp))
  texts.append(str(extract_content(fp)))
  #print(f'{i} process.')

In [8]:
df = pd.DataFrame({'texts': texts, 'pages': target})
df.head()

,texts,pages
0,"['xxQ00_00 xxbob JUL 14 14:48 FR CALIFORNIA SPEEDURY 429 5998 TO 919176631462\n', 'xxeob xxQ01_00\n', 'xxQ01_00 xxbob july 7, 2900\n', 'xxeob xxQ01_00\n', 'xxQ01_00 xxbob Philip Morris, USA\n', '-120 Park Avenue\n', 'xxeob xxQ02_01\n', 'xxQ01_00 xxbob New York, NY 10017-5592\n', 'Dear ina,\n', 'xxeob xxQ03_03\n', 'xxQ02_00 xxbob xxbcet Thank far your time today with our several conversations over the telephone.\n', 'xxecet you regarding Marlboro Racing Experience and\n', 'writing the agreement we that we agreement that for the of the\n', 'SportService. tis my understanding are all in -\n',...",0
1,"['xxQ00_02 xxbob ®\n', 'xxeob xxQ00_01\n', 'xxQ00_01 xxbob Phase\n', 'xxeob xxQ00_03\n', 'xxQ00_00 xxbob Now smoke two packs of the cigarette coded and circle\n', 'Just did . a Number for each pair of phrases\n', 'as you before.\n', 'xxeob xxQ00_00\n', 'xxQ00_00 xxbob CIGARETTES CODED\n', 'xxeob xxQ02_03\n', 'xxQ00_00 xxbob xxbcet Too much menthol taste. | 3 | 2 |3 7] enough menthol taste,\n', 'xxecet Burns too fast, 2-13 14 15 16 Does not burn too fast,\n', '4\n', 'xxbcet Is dry tasting. | 2 [4 {5 [6 [7 } Is not dry tasting,\n', 'xxecet xxbcet Does not have a good taste. | 1 15 6 | | Has ...",1
2,"['xxQ04_02 xxbob BAILOWOLITY\n', 'xxbcet |\n', 'xxecet xxbcet |\n', 'xxecet xxeob xxQ02_02\n', 'xxQ02_02 xxbob xxbcet |\n', 'xxecet xxeob xxQ04_02\n', 'xxQ01_02 xxbob xxbcet |\n', 'xxecet xxbcet |\n', 'xxecet xxbcet |\n', 'xxecet xxbcet |\n', 'xxecet xxbcet |\n', 'xxecet xxeob xxQ04_01\n', 'xxQ04_01 xxbob xxbcet |\n', 'xxecet xxeob xxQ04_01\n', 'xxQ04_01 xxbob xxbcet |\n', 'xxecet xxeob xxQ03_01\n', 'xxQ03_01 xxbob xxbcet |\n', 'xxecet xxeob xxQ04_01\n', 'xxQ03_01 xxbob ——!\n', '—\n', 'xxeob xxQ01_01\n', 'xxQ01_01 xxbob MAN\n', 'xxeob xxQ01_01\n']",1
3,"['xxQ00_00 xxbob THE\n', 'AMERICAN\n', 'TOBACCO\n', 'COMPANY\n', 'xxeob xxQ01_02\n', 'xxQ01_02 xxbob March 30, 1989\n', 'xxeob xxQ01_02\n', 'xxQ01_00 xxbob Mr. Douglass Bomeisler\n', 'Laurence, Charles, Free & Lawson, Inc.\n', '260 Madison Avenue\n', 'New York, New York 10016\n', 'xxeob xxQ02_02\n', 'xxQ02_00 xxbob Re: AMERICAN Advertising Test Creative\n', 'Dear Doug:\n', 'xxeob xxQ02_03\n', ""xxQ02_00 xxbob This will recap the several issues we've discussed regarding an AMERICAN\n"", 'advertising test:\n', 'xxeob xxQ02_01\n', 'xxQ02_00 xxbob I) Background\n', 'xxeob xxQ03_02\n', ""xxQ02_00 ...",1
4,"['xxQ00_01 xxbob xxbcet Tse For Inc.\n', 'xxecet xxbcet 900 THIRD AVENUE\n', 'xxecet xxbcet NEW YORK, 10022\n', 'xxecet xxeob xxQ00_02\n', 'xxQ00_02 xxbob March 19, 1991\n', 'xxeob xxQ01_01\n', 'xxQ01_00 xxbob Dr. C. William Davis\n', 'Div. of Pulmonary Medicine\n', '724 Clinical Sciences Bldg. -7020\n', 'University of North Carolina\n', 'Chapel Hill, NC. 27599\n', 'xxeob xxQ01_01\n', 'xxQ01_00 xxbob Dear Dr. Davis:\n', 'xxeob xxQ02_03\n', 'xxQ01_00 xxbob Thank you for your expression of interest in our program of research support. Iam\n', 'pleased to enclose a statement describing policie...",0


In [9]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [10]:
dls = TextDataLoaders.from_df(df, path=path, text_col='texts', Target ='pages')
dls.show_batch(max_n=3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,"xxbos [ ' xxq00_00 xxbob xxunk \n ' , ' xxeob xxq04_01 \n ' , ' xxq00_00 xxbob { b } xxmaj whether each applicant has \n ' , ' xxunk leaders of that group . \n ' , ' xxbcet 2 xxmaj to determine with xxunk to the \n ' , ' xxecet xxbcet efforts of to ascertain the needs of \n ' , ' xxecet its proposed service \n ' , ' ( a ) xxmaj whether the xxunk ! xxunk \n ' , ' leaders of and in \n ' , ' xxunk . \n ' , ' whether the xxunk xxunk \n ' , ' xxunk xxunk xxunk of xxunk \n ' , ' community of license . \n ' , ' xxbcet 3 . xxmaj to determine whether xxmaj xxunk \n ' , ' xxecet is financially qualified to \n ' , ' xxunk and",1
1,"xxbos [ ' xxq00_00 xxbob xxup martin xxup lipton \n ' , ' herbert xxup wachtell \n ' , ' xxunk xxup xxunk \n ' , ' xxunk xxup w. xxup nussbaum \n ' , ' richard xxup d. xxup xxunk \n ' , ' douglas $ , \n ' , ' steven xxup m. xxup barna \n ' , ' chaim xxup j , xxup fortgang \n ' , ' peter xxup canellos \n ' , ' michael xxup w. xxup schwartz \n ' , ' allan xxup a. xxup martin \n ' , ' xxunk xxup a. xxup bryer \n ' , ' lawrence xxup b , xxup xxunk \n ' , ' robert @. xxup mazur \n ' , ' paul xxup vizcarrondo , xxup jr . \n ' , ' peter xxup g. xxup hein \n ' , ' harold \n ' , ' m ,",1
2,"xxbos [ ' xxq00_00 xxbob xxup martin xxup lipton \n ' , ' herbert xxup m. xxup wachtell \n ' , ' xxunk xxup gewertz \n ' , ' bernard xxup w. xxup nussbaum \n ' , ' richard xxup d , xxup xxunk \n ' , ' douglas xxup liebhafsky \n ' , ' steven xxup m. xxup barna \n ' , ' chaim xxup j. xxup fortgang \n ' , ' peter xxup c , \n ' , ' schwartz \n ' , ' allan xxup al \n ' , ' barry xxup a. xxup bryer \n ' , ' lawrence xxup b , xxup pedowitz \n ' , ' robert xxup b , xxup mazur \n ' , ' paul xxup xxunk , \n ' , ' peter xxup g. xxup hein \n ' , ' harold 5 . \n ' , ' david xxup m. xxup",1


In [11]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5,metrics=[accuracy, Perplexity()]).to_fp16()

In [12]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.708065,0.692034,0.573643,1.997776,00:06


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.619720,0.652262,0.589147,1.919879,00:13
1,0.583561,0.573735,0.724806,1.774883,00:13
2,0.543328,0.428818,0.821705,1.535441,00:13
3,0.507129,0.463919,0.794574,1.590294,00:13


In [13]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.437807,0.498547,0.790698,1.646328,00:06


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.442127,0.350734,0.864341,1.420109,00:13
1,0.437431,0.560695,0.767442,1.751890,00:13
2,0.406804,0.361691,0.872093,1.435755,00:13
3,0.377571,0.357034,0.868217,1.429084,00:13


In [14]:
learn.show_results()

,text,category,category_
0,"xxbos [ ' xxq00_00 xxbob xxbcet xxmaj the | and xxmaj nicotine xxmaj content xxmaj gl \n ' , ' xxecet xxeob xxq01_01 \n ' , ' xxq00_00 xxbob ( 176 ) of \n ' , ' xxunk ‘ ta \n ' , ' long xxunk , or \n ' , ' 2 93 \n ' , ' xxunk ere . 5 \n ' , ' 16 2 \n ' , ' 1 \n ' , ' of \n ' , ' xxunk — of \n ' , ' xxunk , or \n ' , ' it \n ' , ' xxeob xxq02_01 \n ' , ' xxq02_00 xxbob few ane dey xxunk \n ' , ' xxunk \n ' , ' xxeob xxq02_01 \n ' , ' xxq02_00 xxbob of the dated \n ' , ' 5 , 1973 . \n ' , ' james xxup a. xxmaj xxunk",1,1
1,"xxbos [ ' xxq00_00 xxbob “ wlrk xxunk xxmaj server 5 / 28 / 99 \n ' , ' xxeob xxq00_02 \n ' , ' xxq00_00 xxbob i : james xxmaj wilson company : lorillard xxmaj tobacco xxmaj company \n ' , ' xxeob xxq01_00 \n ' , ' xxq00_00 xxbob xxup martin \n ' , ' theodore xxup gewertz \n ' , ' nussbaum \n ' , ' richard xxup d. xxup katcher \n ' , ' xxunk xxup barna \n ' , ' chaim @ xxup fortgang \n ' , ' & , \n ' , ' schwartz \n ' , ' allan \n ' , ' lawrence at xxup bryer \n ' , ' peter \n ' , ' xxunk \n ' , ' david xxup einhorn \n ' , ' xxeob xxq01_00 \n ' , ' xxq01_00 xxbob xxup forrest \n ' , ' xxunk \n",0,0
2,"xxbos [ ' xxq00_00 xxbob xxup martin \n ' , ' herbert xxup m. \n ' , ' theodore xxup gewertz \n ' , ' bernard xxup w. \n ' , ' richard 0 . xxup katcher \n ' , ' xxunk 5 . \n ' , ' steven xxup m. xxup barna \n ' , ' chaim xxup j. xxup fortgang \n ' , ' peter xxup c. xxup canellos \n ' , ' michael xxup w. xxup schwartz \n ' , ' allan xxup a , xxup martin \n ' , ' barry xxup a. xxup bryer \n ' , ' lawrence xxup b. \n ' , ' mazur xxup jr . \n ' , ' paul , xxup vizcarrondo , \n ' , ' peter © . xxup hein \n ' , ' harold xxup novikoff \n ' , ' m. xxup einhorn \n ' , '",1,1
3,"xxbos [ ' xxq00_01 xxbob xxunk \n ' , ' sudol , xxmaj marius 129 - 68 - 8626 \n ' , ' xxeob xxq05_03 \n ' , ' xxq00_00 xxbob 3 . xxmaj xxunk , xxup xxunk . , xxmaj harvey , xxup r. , xxmaj xxunk , xxup xxunk . , xxmaj xxunk , xxup k. , xxmaj xxunk , xxup t. , xxmaj toyoshima , xxup t. , \n ' , ' smith , xxup xxunk . ( 1988 ) xxup xxunk , xxup j. xxunk - xxunk \n ' , ' 4 . xxmaj xxunk , ( 1983 ) xxmaj nature xxunk - xxunk \n ' , ' 5 . xxmaj xxunk , xxup j. , xxmaj xxunk , xxup xxunk . , and xxmaj vogt , xxup xxunk . ( 1981 ) xxmaj xxunk . xxunk . xxmaj sci , xxup usa xxunk \n '",1,1
4,"xxbos [ ' xxq00_00 xxbob 85 / 26 / 99 \n ' , ' xxeob xxq02_00 \n ' , ' xxq01_00 xxbob w. \n ' , ' 9 . \n ' , ' xxunk \n ' , ' xxunk xxup xxunk \n ' , ' a , \n ' , ' a , \n ' , ' robert 5 , \n ' , ' peter \n ' , ' xxunk 4 , \n ' , ' & \n ' , ' 0 . \n ' , ' xxunk xxup xxunk \n ' , ' xxunk \n ' , ' xxunk xxup xxunk \n ' , ' be \n ' , ' mane \n ' , ' xxunk \n ' , ' al \n ' , ' xxeob xxq00_02 \n ' , ' xxq00_00 xxbob xxunk xxup wachtell xxup lipton > xxup xxunk \n ' , ' xxeob xxq02_01 \n '",1,0
5,"xxbos [ ' xxq00_00 xxbob xxunk xxmaj server \n ' , ' xxeob xxq00_01 \n ' , ' xxq00_00 xxbob to : ronald xxmaj xxunk xxup company ! \n ' , ' xxeob xxq02_00 \n ' , ' xxq01_00 xxbob xxup w. \n ' , ' xxunk 3 . \n ' , ' allan xxup al \n ' , ' barry xxup a. \n ' , ' xxunk xxup b , \n ' , ' xxunk xxup jr \n ' , ' xxunk \n ' , ' harold xxup s. \n ' , ' & xxup forrest \n ' , ' edward \n ' , ' & \n ' , ' daniel xxup a. xxup xxunk \n ' , ' andrew xxmaj re \n ' , ' rowe . \n ' , ' michael & xxup xxunk \n ' , ' a. \n ' , ' xxeob xxq02_01 \n '",0,0
6,"xxbos [ ' xxq00_00 xxbob xxunk \n ' , ' xxeob xxq02_00 \n ' , ' xxq01_00 xxbob s \n ' , ' peter © , \n ' , ' w \n ' , ' c. \n ' , ' & \n ' , ' xxunk \n ' , ' a \n ' , ' warren xxmaj be \n ' , ' pe \n ' , ' rowe \n ' , ' xxeob xxq02_01 \n ' , ' xxq01_00 xxbob xxup patricia \n ' , ' & \n ' , ' seth \n ' , ' xxunk \n ' , ' 5 \n ' , ' xxunk \n ' , ' 4 \n ' , ' d. \n ' , ' xxunk \n ' , ' & \n ' , ' xxunk \n ' , ' were xxup xxunk \n ' , ' = \n ' , ' xxeob xxq01_02",1,1
7,"xxbos [ ' xxq00_01 xxbob 3d \n ' , ' sudol , xxmaj marius 129 - 68 - 8626 \n ' , ' xxeob xxq00_03 \n ' , ' xxq00_00 xxbob xxunk mrna and whose xxunk protei

In [16]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path), CategoryBlock),
                 get_items=get_text_files,
                 get_y=parent_label,
                 splitter=GrandparentSplitter(valid_name='test'))

In [20]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.748472,0.690011,0.527132,1.993738,00:06


In [22]:
learn.save('1epoch')

Path('/content/gdrive/MyDrive/Colab Notebooks/Tobacco_LSTM/models/1epoch.pth')

In [23]:
learn = learn.load('1epoch')

We can them finetune the model after unfreezing:

In [24]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,0.718991,0.623577,0.666667,1.865589,00:13
1,0.672988,0.506116,0.779070,1.658835,00:13
2,0.624913,0.624563,0.670543,1.867429,00:13
3,0.584749,0.365599,0.856589,1.441377,00:13
4,0.534674,0.412113,0.779070,1.510005,00:13
5,0.489264,0.337138,0.864341,1.400932,00:13
6,0.444938,0.570249,0.724806,1.768707,00:13
7,0.405039,0.377413,0.852713,1.458507,00:13
8,0.375035,0.352478,0.856589,1.422589,00:13
9,0.352140,0.332117,0.860465,1.393916,00:13


In [25]:
learn.save_encoder('finetuned')

### Training a text classifier

We can gather our data for text classification almost exactly like before:

In [27]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [39]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [40]:
learn = learn.load_encoder('finetuned')

In [41]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.582025,0.623866,0.810078,00:06


In [42]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.371033,0.369717,0.860465,00:07


Then we can unfreeze a bit more, and continue training:

In [43]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.355807,0.530631,0.775194,00:10


In [46]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.272755,0.301056,0.875969,00:13
1,0.261317,0.340314,0.860465,00:13
2,0.255657,0.327053,0.864341,00:13
